In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, TweedieRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

In [2]:
in_sample_df = pd.read_csv('../Cleaned Datasets/in_sample_cleaned.csv')
out_sample_df = pd.read_csv('../Cleaned Datasets/out_sample_cleaned.csv')

In [3]:
# Creating Previous n-Day Returns
estimation_window = 5

def create_previous_day_returns(permno, in_sample_df, out_sample_df, estimation_window):
    # Iterating over each unique stock (PERMNO) in the in-sample data
    stock_in = in_sample_df[in_sample_df['PERMNO'] == permno].sort_values('DlyCalDt')
    stock_out = out_sample_df[out_sample_df['PERMNO'] == permno].sort_values('DlyCalDt')

    combined_df = pd.concat([stock_in, stock_out]).sort_values('DlyCalDt').reset_index(drop=True)

    # Create lag features
    for lag in range(1, estimation_window + 1):
        combined_df[f'{lag}DayRet'] = combined_df['ExcessReturn'].shift(lag)
    
    return combined_df

total_stock_df = pd.DataFrame()

for permno in in_sample_df['PERMNO'].unique():
    combined_df = create_previous_day_returns(permno, in_sample_df, out_sample_df, estimation_window)
    
    # Append the results to a master DataFrame
    total_stock_df = pd.concat([total_stock_df,combined_df], ignore_index=True)
    
total_stock_df.sort_values(['DlyCalDt','Ticker'], inplace=True)
total_stock_df.dropna(inplace=True)
total_stock_df['DlyCalDt'] = pd.to_datetime(total_stock_df['DlyCalDt'])

In [4]:
total_stock_df

,PERMNO,CUSIP,Ticker,PERMCO,SICCD,DlyCalDt,DlyPrc,DlyCap,DlyRet,DlyRetx,...,DlyPrcVol,ShrOut,TDYLD,MktCap,ExcessReturn,1DayRet,2DayRet,3DayRet,4DayRet,5DayRet
162219,79057,G0070K10,ACL,29796,6331,2000-01-10,18.6875,3.626478e+06,0.031034,0.031034,...,3.011491e+07,194059.0,0.000147,3.626478e+06,0.030887,0.123885,0.057231,-0.058063,-0.007810,-0.022618
205892,57904,00105510,AFL,92,6321,2000-01-10,42.3750,1.127675e+07,-0.059639,-0.059639,...,5.854950e+07,266118.0,0.000147,1.127675e+07,-0.059786,0.019656,0.020056,-0.005887,-0.026683,-0.051802
24961,66800,02687410,AIG,137,6711,2000-01-10,107.5625,1.665327e+08,-0.017694,-0.017694,...,2.337977e+08,1548241.0,0.000147,1.665327e+08,-0.017841,0.074701,0.030521,0.002070,-0.051259,-0.038874
149741,79323,02000210,ALL,29870,6331,2000-01-10,23.9375,1.927917e+07,-0.022959,-0.022959,...,6.934694e+07,805396.0,0.000147,1.927917e+07,-0.023106,0.036891,-0.005409,0.043808,-0.029480,-0.026120
286999,85592,00163T10,AMB,32135,6798,2000-01-10,20.1250,1.742362e+06,0.006250,0.006250,...,3.183775e+06,86577.0,0.000147,1.742362e+06,0.006103,0.018962,0.019335,-0.012969,-0.015920,-0.006416
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286993,28388,92904210,VNO,21874,6798,2024-12-31,42.0400,8.014884e+06,0.015950,0.015950,...,3.492200e+07,190649.0,0.000112,8.014884e+06,0.015838,-0.009926,-0.020275,0.005307,0.008686,0.012894
255798,75819,92276F10,VTR,10302,6798,2024-12-31,58.8900,2.532924e+07,0.014186,0.006495,...,1.359223e+08,430111.0,0.000112,2.532924e+07,0.014074,-0.006568,-0.008363,0.001741,0.005657,0.002778
243320,65947,95040Q10,WELL,2105,6513,2024-12-31,126.0300,7.847762e+07,0.008886,0.008886,...,3.333996e+08,622690.0,0.000112,7.847762e+07,0.008774,-0.000755,-0.011109,-0.001695,0.007609,0.007748
6238,38703,94974610,WFC,21305,6021,2024-12-31,70.2400,2.338634e+08,-0.002414,-0.002414,...,4.939006e+08,3329491.0,0.000112,2.338634e+08,-0.002526,-0.009959,-0.009169,0.002260,0.014772,0.002728


### Approach 1 


The model is trained once for each stock, and the overall results are calculated 

In [5]:
'''from sklearn.linear_model import LassoCV

estimation_window = 5
out_sample_start = '2016-01-01'
out_sample_start = pd.to_datetime(out_sample_start)
in_sample_start = '2000-01-01'
in_sample_start = pd.to_datetime(in_sample_start)

alpha = 0.01
results = []

def rolling_window_linear_model (permno, in_sample_df, out_sample_df, linear_model, estimation_window=estimation_window, alpha=alpha):
    
    stock_in = in_sample_df[in_sample_df['PERMNO'] == permno].sort_values('DlyCalDt')
    stock_out = out_sample_df[out_sample_df['PERMNO'] == permno].sort_values('DlyCalDt')
    #ticker = stock_in['Ticker'].iloc[0]

    # Merge in-sample and out-of-sample data
    full_stock_df = pd.concat([stock_in, stock_out]).sort_values('DlyCalDt').reset_index(drop=True)

    # Create lag features
    for lag in range(1, estimation_window + 1):
        full_stock_df[f'{lag}DayReturn'] = full_stock_df['ExcessReturn'].shift(lag)

    # Creating the next day's excess return as the target variable
    full_stock_df['NextDayExcessReturn'] = full_stock_df['ExcessReturn'].shift(-1)
    full_stock_df.dropna(inplace=True)
    
    # Training data for the model. 
    # Training data is the data before the out-of-sample start date and includes the lagged features.
    X_train = full_stock_df.loc[(full_stock_df['DlyCalDt'] < out_sample_start),  
                                [f'{lag}DayReturn' for lag in range(1, estimation_window + 1)]
                                ]
    
    # Target variable for the model.
    # Target variable is the next day's excess return. 
    # This is the y variable of training dataset 
    y_train = full_stock_df.loc[full_stock_df['DlyCalDt'] < out_sample_start, 
                                'NextDayExcessReturn']

    if linear_model == "Lasso":
        model = Lasso(alpha=alpha)
    elif linear_model == "Ridge":
        model = Lasso(alpha=alpha)
    
    model.fit(X_train, y_train)  
    
    X_test = full_stock_df.loc[full_stock_df['DlyCalDt'] >= out_sample_start, 
                              [f'{lag}DayReturn' for lag in range(1, estimation_window + 1)]]

    y_test = full_stock_df.loc[full_stock_df['DlyCalDt'] >= out_sample_start,
                              'NextDayExcessReturn']
    
    
    # Predict the actual next day (same input X)
    y_pred = model.predict(X_test)

    combined = X_test.merge(y_test, left_index=True, right_index=True).reset_index(drop=True)
    y_pred = pd.DataFrame(y_pred)
    y_pred.columns = ['PredictedReturns']
    combined = combined.merge(y_pred, left_index=True, right_index=True)
    combined['PERMNO'] = permno
    
    return combined
    
results = pd.DataFrame()
for permno in in_sample_df['PERMNO'].unique():
    
    combined_df = rolling_window_linear_model(permno, in_sample_df, out_sample_df, "Lasso", estimation_window=estimation_window, alpha=alpha)

    # Append the results to a master DataFrame
    results = pd.concat([results,combined_df], ignore_index=True)

mse = mean_squared_error(results['NextDayExcessReturn'], results['PredictedReturns'])
r2 = r2_score(results['NextDayExcessReturn'], results['PredictedReturns'])

print("R2 value for Lasso of all 50 stocks is:",r2)
print("MSE for Lasso of all 50 stocks is:",mse)'''


'from sklearn.linear_model import LassoCV\n\nestimation_window = 5\nout_sample_start = \'2016-01-01\'\nout_sample_start = pd.to_datetime(out_sample_start)\nin_sample_start = \'2000-01-01\'\nin_sample_start = pd.to_datetime(in_sample_start)\n\nalpha = 0.01\nresults = []\n\ndef rolling_window_linear_model (permno, in_sample_df, out_sample_df, linear_model, estimation_window=estimation_window, alpha=alpha):\n\n    stock_in = in_sample_df[in_sample_df[\'PERMNO\'] == permno].sort_values(\'DlyCalDt\')\n    stock_out = out_sample_df[out_sample_df[\'PERMNO\'] == permno].sort_values(\'DlyCalDt\')\n    #ticker = stock_in[\'Ticker\'].iloc[0]\n\n    # Merge in-sample and out-of-sample data\n    full_stock_df = pd.concat([stock_in, stock_out]).sort_values(\'DlyCalDt\').reset_index(drop=True)\n\n    # Create lag features\n    for lag in range(1, estimation_window + 1):\n        full_stock_df[f\'{lag}DayReturn\'] = full_stock_df[\'ExcessReturn\'].shift(lag)\n\n    # Creating the next day\'s excess re

### Approach 2

The model is trained for all 50 stocks, and the results are calculated. The model is retrained every n days (tunable) by expanding the training dates

In [9]:

def rolling_window(total_stock_df, input_model, estimation_window=estimation_window, alpha=0.001):
    y_preds = []
    y_actuals = []
    predictions = []
    out_sample_start = '2016-01-01'
    out_sample_start = pd.to_datetime(out_sample_start)
    unique_dates = sorted(total_stock_df['DlyCalDt'].unique())

    # Filter only dates >= out_sample_start
    prediction_dates = [d for d in unique_dates if d >= out_sample_start]

    for current_date in prediction_dates:
        # Get N trading days before current_date
        train_end = current_date - pd.Timedelta(days=1)
        train_start_idx = total_stock_df['DlyCalDt'] <= train_end
        window_data = total_stock_df[train_start_idx]
        
        # Ensure we get exactly N trading days (not calendar days)
        last_n_days = window_data['DlyCalDt'].drop_duplicates().nlargest(estimation_window)
        train_dates = last_n_days.min(), last_n_days.max()

        train_data = window_data[window_data['DlyCalDt'].isin(last_n_days)]

        if train_data.empty:
            continue

        # Get test data for current date
        test_data = total_stock_df[total_stock_df['DlyCalDt'] == current_date]
        if test_data.empty:
            continue

        X_train = train_data[[f'{lag}DayRet' for lag in range(1, estimation_window + 1)]].values
        X_test = test_data[[f'{lag}DayRet' for lag in range(1, estimation_window + 1)]].values
        
        y_train = train_data['ExcessReturn']
        y_test = test_data['ExcessReturn']

        # Initialize model
        if input_model == 'Ridge':
            model = Ridge(alpha=alpha)
        elif input_model == 'OLS':
            model = LinearRegression()
        elif input_model == 'ElasticNet':
            model = ElasticNet(alpha=alpha)
        elif input_model == 'Lasso':
            model = Lasso(alpha=alpha)
        elif input_model == 'GLM':
            model = TweedieRegressor(power=0, alpha=alpha)
        elif input_model == 'RF':
            model = RandomForestRegressor(n_estimators=500, min_samples_leaf=10, max_depth=5, max_features=0.5, n_jobs=-1)
        elif input_model == 'GBRT':
            model = GradientBoostingRegressor(n_estimators=300, learning_rate=0.03, max_depth=3, subsample=0.8, loss='huber')
        elif input_model == 'NN1':
            model = MLPRegressor(hidden_layer_sizes=(16,), max_iter=1000, activation='relu', solver='adam', learning_rate='adaptive', early_stopping=True)
        elif input_model == 'NN2':
            model = MLPRegressor(hidden_layer_sizes=(32,16), max_iter=1000, activation='relu', solver='adam', learning_rate='adaptive', early_stopping=True)
        elif input_model == 'NN3':
            model = MLPRegressor(hidden_layer_sizes=(64,32,16), max_iter=1000, activation='relu', solver='adam', learning_rate='adaptive', early_stopping=True)
        elif input_model == 'NN4':
            model = MLPRegressor(hidden_layer_sizes=(64,32,16,8), max_iter=1000, activation='relu', solver='adam', learning_rate='adaptive', early_stopping=True)
        elif input_model == 'NN5':
            model = MLPRegressor(hidden_layer_sizes=(64,32,16,8,4), max_iter=1000, activation='relu', solver='adam', learning_rate='adaptive', early_stopping=True)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        predictions.extend([
            {'Date': current_date, 'PERMNO': permno, 'Actual': actual, 'Predicted': pred}
            for permno, actual, pred in zip(test_data['PERMNO'], y_test.values, y_pred)
        ])
        y_preds.extend(y_pred)
        y_actuals.extend(y_test.values)

    y_actuals_np = np.array(y_actuals)
    y_preds_np = np.array(y_preds)
    sign_match = np.sign(y_actuals_np) == np.sign(y_preds_np)

    # Up and Down masks
    up_da = sign_match[y_actuals_np > 0].mean() if np.any(y_actuals_np > 0) else np.nan
    down_da = sign_match[y_actuals_np < 0].mean() if np.any(y_actuals_np < 0) else np.nan
    mse = mean_squared_error(y_actuals, y_preds)
    r2 = r2_score(y_actuals, y_preds)
    mae = mean_absolute_error(y_actuals, y_preds)
    da = (np.sign(y_actuals) == np.sign(y_preds)).mean()

    return mse, r2, mae, da, up_da, down_da, predictions

In [10]:
# Implementing the rolling window for different models
models = ['Ridge', 'OLS', 'ElasticNet', 'Lasso', 'GLM', 'RF', 'NN1', 'NN2', 'NN3', 'NN4', 'NN5', 'GBRT']
results = []
wide_preds_df = pd.DataFrame()
estimation_window = 5

for alpha in [0.001]:
    print(f"Running models with alpha: {alpha}")
    for model in models: 
        
        mse, r2, mae, da, up_da, down_da, predictions = rolling_window(total_stock_df, input_model=model, estimation_window=estimation_window, alpha=alpha)
        print(f"Model: {model}, MSE: {mse:.7f}, R2: {r2:.7f}, MAE: {mae:.7f}, Direction Accuracy: {da:.7f}, Up Directional Accuracy: {up_da:.7f}, Down Directional Accuracy: {down_da:.7f}")
 
        results.append({
            'Model': model,
            'MSE': mse,
            'R2': r2,
            'MAE': mae,
            'Direction Accuracy': da,
            'Up Directional Accuracy': up_da,
            'Down Directional Accuracy': down_da
        })
        
        # Convert predictions list to DataFrame
        pred_df = pd.DataFrame(predictions)
        pred_df.rename(
            columns={'Date': 'DlyCalDt', 'Actual': 'ExcessReturn', 'Predicted': f'{model}'},
            inplace=True
        )

        # Merge predictions into wide_preds_df
        if wide_preds_df.empty:
            wide_preds_df = pred_df[['PERMNO', 'DlyCalDt', 'ExcessReturn', f'{model}']]
        else:
            wide_preds_df = wide_preds_df.merge(
                pred_df[['PERMNO', 'DlyCalDt', f'{model}']],
                on=['PERMNO', 'DlyCalDt'],
                how='left'
            )


# Convert to DataFrame
results_df = pd.DataFrame(results)
results_df.sort_values(by='Model', inplace=True)
results_df.to_csv('model_results.csv (5-window).csv', index=False)
# Save predictions to CSV
wide_preds_df.sort_values(['DlyCalDt', 'PERMNO'], inplace=True)
wide_preds_df.to_csv('model_predictions.csv (5-window).csv', index=False)

Running models with alpha: 0.001
Model: Ridge, MSE: 0.0005988, R2: -0.7398323, MAE: 0.0150735, Direction Accuracy: 0.5007473, Up Directional Accuracy: 0.5464776, Down Directional Accuracy: 0.4500535
Model: OLS, MSE: 0.0006050, R2: -0.7576538, MAE: 0.0151652, Direction Accuracy: 0.5009609, Up Directional Accuracy: 0.5456992, Down Directional Accuracy: 0.4513666
Model: ElasticNet, MSE: 0.0004141, R2: -0.2031791, MAE: 0.0129242, Direction Accuracy: 0.5012900, Up Directional Accuracy: 0.5932175, Down Directional Accuracy: 0.3993847
Model: Lasso, MSE: 0.0004019, R2: -0.1677013, MAE: 0.0128717, Direction Accuracy: 0.5019929, Up Directional Accuracy: 0.5932513, Down Directional Accuracy: 0.4008292
Model: GLM, MSE: 0.0004341, R2: -0.2612795, MAE: 0.0131242, Direction Accuracy: 0.4971352, Up Directional Accuracy: 0.5854670, Down Directional Accuracy: 0.3992159
Model: RF, MSE: 0.0004555, R2: -0.3234458, MAE: 0.0137989, Direction Accuracy: 0.4978559, Up Directional Accuracy: 0.5530097, Down Direc

C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaa

Model: NN1, MSE: 0.0005726, R2: -0.6636395, MAE: 0.0155161, Direction Accuracy: 0.4941281, Up Directional Accuracy: 0.5284044, Down Directional Accuracy: 0.4561315
Model: NN2, MSE: 0.0005122, R2: -0.4882531, MAE: 0.0144323, Direction Accuracy: 0.5002491, Up Directional Accuracy: 0.5350718, Down Directional Accuracy: 0.4616467
Model: NN3, MSE: 0.0004979, R2: -0.4465682, MAE: 0.0140720, Direction Accuracy: 0.5004093, Up Directional Accuracy: 0.5561234, Down Directional Accuracy: 0.4386478


C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaa

Model: NN4, MSE: 0.0004859, R2: -0.4116493, MAE: 0.0140182, Direction Accuracy: 0.4972776, Up Directional Accuracy: 0.5332273, Down Directional Accuracy: 0.4574259


C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\amaa

Model: NN5, MSE: 0.0004864, R2: -0.4131144, MAE: 0.0143832, Direction Accuracy: 0.4974110, Up Directional Accuracy: 0.5467991, Down Directional Accuracy: 0.4426623
Model: GBRT, MSE: 0.0005540, R2: -0.6095156, MAE: 0.0149504, Direction Accuracy: 0.4999377, Up Directional Accuracy: 0.5429408, Down Directional Accuracy: 0.4522671
